In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import sys
import random
import os

from IPython import display
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split


from functions.parse_data import synth_dataloader
from multivariate_quantile_regression.network_model import QuantileNetwork

from cot_train.utils import MLP5

/tmp/ipykernel_23796/4037808937.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-18 13:22:51.882024: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-18 13:22:51.916388: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 13:22

In [3]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage and check:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)
device = tensor.device

CUDA is available. Using GPU.
Tensor is on device: cuda:0


In [4]:
#load all data
df=synth_dataloader(path_name='SMHIdata',drop_cols=True)

In [5]:
#Load training, val and test data for indices
traindata = pd.read_csv('cot_train/data/synthetic-cot-data/train_df.csv', index_col=[0])
valdata = pd.read_csv('cot_train/data/synthetic-cot-data/val_df.csv', index_col=[0])
testdata = pd.read_csv('cot_train/data/synthetic-cot-data/test_df.csv', index_col=[0])

#Find indices for each
trainidx = traindata['index'].to_numpy()
validx = valdata['index'].to_numpy()
testidx = testdata['index'].to_numpy()

In [6]:
#Find X and y
X_cols = ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
          'Cloud_B07','Cloud_B08','Cloud_B08A','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12']
y_cols = ['Clear_B02','Clear_B03','Clear_B04','Clear_B05','Clear_B06',
          'Clear_B07','Clear_B08','Clear_B08A','Clear_B09','Clear_B10','Clear_B11','Clear_B12']

X=df[X_cols].to_numpy()
y=df[y_cols].to_numpy()

In [16]:
#load pretrained COT-prediction models
model0 = MLP5(12, 1, apply_relu=True)
model0.load_state_dict(torch.load('cot_train/log/2024-03-14_09-24-00/model_it_2000000', map_location=device))
#model0.to(device)
#model0.eval()

COT_model0 = torch.load('pytorch_models/Loop_learning_rate/0.003/model0/model_file')
COT_model1 = torch.load('pytorch_models/Loop_learning_rate/0.003/model1/model_file')
COT_model2 = torch.load('pytorch_models/Loop_learning_rate/0.003/model2/model_file')
COT_model3 = torch.load('pytorch_models/Loop_learning_rate/0.003/model3/model_file')
COT_model4 = torch.load('pytorch_models/Loop_learning_rate/0.003/model4/model_file')

#Find real COT values
trueCOT = df['COT'].to_numpy()

#Add noise to X and predict COT_pred w/ each model and take mean
X_noisy = X + np.random.randn(np.shape(X)[0],np.shape(X)[1]) * np.mean(X,axis=0)*0.03
X_train_mu = np.mean(X,axis=0)
X_train_std= np.std(X,axis=0)
X_n = (X_noisy-X_train_mu)/X_train_std
tX_n = torch.Tensor(X_n).to(device)

COT_pred0 = COT_model0.predict(X_noisy)[:,:,4]
COT_pred1 = COT_model1.predict(X_noisy)[:,:,4]
COT_pred2 = COT_model2.predict(X_noisy)[:,:,4]
COT_pred3 = COT_model3.predict(X_noisy)[:,:,4]
COT_pred4 = COT_model4.predict(X_noisy)[:,:,4]
COT_pred = COT_pred0
#COT_pred = (COT_pred0+COT_pred1+COT_pred2+COT_pred3+COT_pred4)/5

COT_pred0_smhi = 50*model0(tX_n).cpu().detach().numpy()


#print different error metrics
print('Stats for our alg:')
print('MAE = ' +str(np.mean(np.abs(COT_pred[:,0]-trueCOT))))
print('RMSE = ' + str(np.sqrt(mean_squared_error(COT_pred[:,0],trueCOT))))
criterion = nn.L1Loss().to(device)
print('L1 loss = ' + str(criterion(torch.tensor(COT_pred[:,0],dtype=torch.float,device=device),torch.tensor(trueCOT,dtype=torch.float,device=device)).cpu().detach().numpy().item()))

print('Stats for Aleksis alg:')
print('MAE = ' +str(np.mean(np.abs(COT_pred0_smhi[:,0]-trueCOT))))
print('RMSE = ' + str(np.sqrt(mean_squared_error(COT_pred0_smhi[:,0],trueCOT))))
criterion = nn.L1Loss().to(device)
print('L1 loss = ' + str(criterion(torch.tensor(COT_pred0_smhi[:,0],dtype=torch.float,device=device),torch.tensor(trueCOT,dtype=torch.float,device=device)).cpu().detach().numpy().item()))


Stats for our alg:
MAE = 2.080872156246265
RMSE = 4.303570789316039
L1 loss = 2.080872058868408
Stats for Aleksis alg:
MAE = 2.3033979227756647
RMSE = 4.2616078193695675
L1 loss = 2.3033976554870605


In [32]:
#Add COT-estimation to X
X_wcot = np.concatenate((X,COT_pred),axis=1)
X_test = X_wcot[testidx,:]
y_test = y[testidx,:]

#Add noise to X_test
X_test = X_test + np.random.randn(np.shape(X_test)[0],np.shape(X_test)[1]) * np.mean(X_wcot,axis=0)*0.03

np.shape(y_test)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 200000 and the array at index 1 has size 20000

In [ ]:
quantiles=np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
no_nodes=100
est= np.where(quantiles==0.5)[0].item()

sequence= lambda: nn.Sequential(
    nn.Linear(len(X_wcot[0,:]),no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

In [ ]:
val_size=0.1
lr_s=[0.003]
num_models=5 #Set number of ensambles
batch_size=500
nepochs=1000
lr=0.003
noise_ratio = 0.03
early_break=True

looped_metric="Loop_learning_rate_w_COT"
main_filepath='pytorch_models/'+looped_metric

model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index',looped_metric,'MSE','PSNR','R2_score','Mean_Quantile_Loss','Quant_Rate','Quantiles'])
for lr in lr_s:

    
    models=[QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]
    preds_total=[]
    for i,model in enumerate(models):       

        model.fit(X_wcot,y, 
            train_indices=trainidx, 
            validation_indices=validx, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break)
        
        filepath=main_filepath+'/'+str(lr)+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')

        preds = model.predict(X_test)
        if i==0:
            preds_total=preds
        else:
            preds_total=preds_total+preds

        mse=mean_squared_error(y_test,preds[:,:,est])
        psnr=QuantileNetwork.PSNR(y_test,preds[:,:,est])
        r2=r2_score(y_test,preds[:,:,est])
        mean_quantile=QuantileNetwork.mean_marginal_loss(y_test,preds,quantiles)
        quant_rate=QuantileNetwork.quant_rate(y_test,preds)

        tmp_metrics=pd.DataFrame(data=[[False,i,lr,mse,psnr,r2,mean_quantile,quant_rate,quantiles]],columns=['Ensemble_mean','Ensemble_index',looped_metric,'MSE','PSNR','R2_score','Mean_Quantile_Loss','Quant_Rate','Quantiles'])
        model_metrics=pd.concat([model_metrics,tmp_metrics])
    
    preds_total=preds_total/num_models

    mse=mean_squared_error(y_test,preds_total[:,:,est])
    psnr=QuantileNetwork.PSNR(y_test,preds_total[:,:,est])
    r2=r2_score(y_test,preds_total[:,:,est])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test,preds_total,quantiles)
    quant_rate=QuantileNetwork.quant_rate(y_test,preds_total)

    tmp_metrics=pd.DataFrame(data=[[True,np.nan,lr,mse,psnr,r2,mean_quantile,quant_rate,quantiles]],columns=['Ensemble_mean','Ensemble_index',looped_metric,'MSE','PSNR','R2_score','Mean_Quantile_Loss','Quant_Rate','Quantiles'])
    model_metrics=pd.concat([model_metrics,tmp_metrics])


model_metrics=model_metrics.reset_index(drop=True)
model_metrics.to_csv(main_filepath+'/model_metrics.csv',index=False)

Epoch 361


Batch number: 100%|██████████| 320/320 [00:00<00:00, 360.12it/s]


Training loss [8.485631] Validation loss [8.461695]
Epoch 362


Batch number: 100%|██████████| 320/320 [00:00<00:00, 364.24it/s]

Training loss [8.449384] Validation loss [8.564287]
Epoch 363


...

Batch number: 100%|██████████| 320/320 [00:00<00:00, 362.78it/s]


Training loss [8.487512] Validation loss [8.542857]
Epoch 370


Batch number: 100%|██████████| 320/320 [00:00<00:00, 348.42it/s]

Training loss [8.459714] Validation loss [8.434978]


---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 270
With validation loss: 8.409407615661621


In [ ]:
model_metrics

,Ensemble_mean,Ensemble_index,Loop_learning_rate_w_COT,MSE,PSNR,R2_score,Mean_Quantile_Loss,Quant_Rate,Quantiles
0,False,0,0.003,0.007889,21.060109,0.815219,1.770330,"[0.11910416666666666, 0.21740833333333334, 0.3...","[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]"
1,False,1,0.003,0.007770,21.126184,0.817985,1.757368,"[0.09772916666666667, 0.18453333333333333, 0.2...","[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]"
2,False,2,0.003,0.007859,21.076680,0.816533,1.762746,"[0.106625, 0.198825, 0.30085, 0.39775416666666...","[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]"
3,False,3,0.003,0.007743,21.140856,0.818673,1.763871,"[0.0999, 0.1938125, 0.30750833333333333, 0.403...","[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]"
4,False,4,0.003,0.007702,21.164437,0.819779,1.760423,"[0.09139166666666666, 0.18505, 0.2847791666666...","[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]"
5,True,NaN,0.003,0.007381,21.349001,0.827284,1.697924,"[0.08546666666666666, 0.17257916666666667, 0.2...","[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]"


In [35]:
testidx

array([ 33041, 146492, 150424, ..., 123473,  65089,  52290])